# Results
Solving the  [Tennis](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Learning-Environment-Examples.md#tennis) environment requires the training of two separate agents, that need to collaborate with each other (do not let the incomming ball hit the ground) and, at the same time, compete with each other (maximizing the number of scores through multiple games).
A simple extension of single agent RL by independently training the two agents does not work very well because the agents are independently updating their policies as learning progresses. And this causes the environment to appear non-stationary from the viewpoint of any one agent. While we can have non-stationary Markov processes, the convergence guarantees offered by many RL algorithms such as Q-learning requires stationary environments. 

A popular algorithm for solving an environment, where multiple agents are coordinating to complete tasks with only local information is the Multi-agent DDPG (MADDPG) ([Lowe et al., 2017](https://arxiv.org/pdf/1706.02275.pdf)).
The algorithm is an extension of [DDPG](https://lilianweng.github.io/lil-log/2018/04/08/policy-gradient-algorithms.html#ddpg).
## An Overview of the MADDPG Algorithm
MADDPG extends DDPG to an environment where multiple agents are coordinating to complete tasks with only local information. In the viewpoint of one agent, the environment is non-stationary as policies of other agents are quickly upgraded and remain unknown. 

MADDPG is an actor-critic model redesigned particularly for handling such a changing environment and interactions between agents.

The problem can be formalized in the multi-agent version of MDP, also
known as *Markov games*. Say, there are N agents in total with a set of
states $\mathcal{S}$. Each agent owns a set of possible action,
$\mathcal{A}_1, \dots, \mathcal{A}_N$, and a set of observation,
$\mathcal{O}_1, \dots, \mathcal{O}_N$. The state transition function
involves all states, action and observation spaces
$\mathcal{T}: \mathcal{S} \times \mathcal{A}_1 \times \dots \mathcal{A}_N \mapsto \mathcal{S}$.
Each agent’s stochastic policy only involves its own state and action:
$\pi_{\theta_i}: \mathcal{O}_i \times \mathcal{A}_i \mapsto [0, 1]$, a
probability distribution over actions given its own observation, or a
deterministic policy:
$\mu_{\theta_i}: \mathcal{O}_i \mapsto \mathcal{A}_i$.

Let $\vec{o} = {o_1, \dots, o_N}$, $\vec{\mu} = {\mu_1, \dots, \mu_N}$
and the policies are parameterized by
$\vec{\theta} = {\theta_1, \dots, \theta_N}$.

The critic in MADDPG learns a centralized action-value function
$Q^\vec{\mu}_i(\vec{o}, a_1, \dots, a_N)$ for the i-th agent, where
$a_1 \in \mathcal{A}_1, \dots, a_N \in \mathcal{A}_N$ are actions of all
agents. Each $Q^\vec{\mu}_i$ is learned separately for $i=1, \dots, N$
and therefore multiple agents can have arbitrary reward structures,
including conflicting rewards in a competitive setting. Meanwhile,
multiple actors, one for each agent, are exploring and upgrading the
policy parameters $\theta_i$ on their own.

**Actor update**:

$$\nabla_{\theta_i} J(\theta_i) = \mathbb{E}_{\vec{o}, a \sim \mathcal{D}} [\nabla_{a_i} Q^{\vec{\mu}}_i (\vec{o}, a_1, \dots, a_N) \nabla_{\theta_i} \mu_{\theta_i}(o_i) \rvert_{a_i=\mu_{\theta_i}(o_i)} ]$$

Where $\mathcal{D}$ is the memory buffer for experience replay,
containing multiple episode samples
$(\vec{o}, a_1, \dots, a_N, r_1, \dots, r_N, \vec{o}')$ — given current
observation $\vec{o}$, agents take action $a_1, \dots, a_N$ and get
rewards $r_1, \dots, r_N$, leading to the new observation $\vec{o}'$.

**Critic update**:

$$% <![CDATA[
\begin{aligned}
\mathcal{L}(\theta_i) &= \mathbb{E}_{\vec{o}, a_1, \dots, a_N, r_1, \dots, r_N, \vec{o}'}[ (Q^{\vec{\mu}}_i(\vec{o}, a_1, \dots, a_N) - y)^2 ] & \\
\text{where } y &= r_i + \gamma Q^{\vec{\mu}'}_i (\vec{o}', a'_1, \dots, a'_N) \rvert_{a'_j = \mu'_{\theta_j}} & \scriptstyle{\text{; TD target!}}
\end{aligned} %]]>$$

where $\vec{\mu}'$ are the target policies with delayed softly-updated parameters.

If the policies $\vec{\mu}$ are unknown during the critic update, we can ask each agent to learn and evolve its own approximation of others’ policies. Using the approximated policies, MADDPG still can learn efficiently although the inferred policies might not be accurate.

To mitigate the high variance triggered by the interaction between competing or collaborating agents in the environment, MADDPG proposed one more element - policy ensembles:

1. Train K policies for one agent;
2. Pick a random policy for episode rollouts;
3. Take an ensemble of these K policies to do gradient update.

In summary, MADDPG added three additional ingredients on top of DDPG to make it adapt to the multi-agent environment:

- Centralized critic + decentralized actors;
- Actors are able to use estimated policies of other agents for learning;
- Policy ensembling is good for reducing variance.
<div>
<img src="./pics/MADDPG-training-diagram.png" width="400"/>
</div>
<center>The architecture design of MADDPG. (Image source: Lowe et al., 2017)<center>

## Training the Agent
Training was repeated three times using different frequencies of updating the local networks with the target ones during an episode.
That is, performing the target network update after 3, 5, 10 steps (see below the hyper parameter `update_every`)

![Update local NN with taget NN after 3 steps](./pics/update_target_every_3_steps.png)
![Update local NN with taget NN after 5 steps](./pics/update_target_every_5_steps.png)
![Update local NN with target NN after 10 steps](./pics/update_target_every_10_steps.png)

Training can be considered as completed when the average score reaches a maximum value through a certain number of episodes. 
That is, for `update_every` equal to `5`, at about after `1750` episodes and for `update_every` equal to `10`, after `1500`.
In both cases the average score reaches a maximum of about `2`. 
It does not make sense to proceed with training after the maximum average score is reached:
- There is no further increase on the average score. 
- The bigger the average score the longer an episode. This will lead to excessive long training times. 
 

### Actor and Critic Neural Networks

The actor network takes state as input and returns the action, whereas the critic network takes state and action as input and returns the value. 
Both, actor and critic use two neural networks (NN): local and target that are shared between the agents that model the two players. 
The local networks are trained by sampling experiences (S;A;R;S') from replay buffer and minimising the loss function. 
The ADAM optimizer is used for minimizing the _loss_.

### Hyper Parameters:
    buffer_size = int(1e5)  # replay buffer size
    batch_size = 128        # mini-batch size
    gamma = 0.99            # reward discount factor
    tau = 1e-3              # for soft update of target parameters
    lr_actor = 1e-5         # actor's learning rate
    lr_critic = 1e-4        # critic's learning rate
    weight_decay = 0        # NN weight decay (L2 penalty)
    update_every = 10       # number of steps before updating the agent's target networks

### Neural Network Configurations
### Actor's Local & Target DQN
1. (  24, 128) - input layer: state space dimension
2. ( 128, 128) - hidden layer
3. ( 128, 128) - hidden layer
4. ( 128,   2) - output layer: action space dimension 

`Leaky ReLU` activation function in the hidden layers; `tanh` activation function in the output layer (the action probabilities)

### Critics's Local & Target DQN
1. (24*2, 128) - input layer: (state space dimension) * (action space dimension)
2. (128, 128) - hidden layer
3. (128, 128) - hidden layer
4. (128, 1) - output layer: Q-value dimension (scalar)

`Leaky ReLU` activation function in the hidden layers; `linear` activation function in the output layer (the Q value) 

### Future Ideas for Improving the Agent's Performance
- The field of Multi-Agent DRLND is a new and evolving one. There is recent research regarding variance reduction in DDPG algorithms that can be also applied in a multi-agent setting. 
It would be interesting to see if [Variance Reduction for Policy Gradient with Action-Dependent Factorized Baselines](https://openreview.net/forum?id=H1tSsb-AW) can be applied in this project.
- Try more advanced algorithms; e.g. [M3DDPG](https://people.eecs.berkeley.edu/~russell/papers/aaai19-marl.pdf) 
